In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#from pyspark.sql.functions import explode,col
from pyspark.sql.types import *

In [2]:
spark =  SparkSession.builder.getOrCreate()

In [3]:
hdfs_path="/input_files"

In [4]:
movies_df=spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/movies.csv")

ratings_df=spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/ratings.csv")
ratings_df1 = ratings_df.withColumn("timestamp",from_unixtime(col("timestamp")))

tags_df=spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/tags.csv")
tags_df1 = tags_df.withColumn("timestamp",from_unixtime(col("timestamp")))

In [5]:
movies_df.show(truncate=False)
ratings_df1.show(truncate=False)
tags_df1.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [6]:
ratings_df.show(truncate=False)
tags_df.show(truncate=False)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |1      |4.0   |964982703|
|1     |3      |4.0   |964981247|
|1     |6      |4.0   |964982224|
|1     |47     |5.0   |964983815|
|1     |50     |5.0   |964982931|
|1     |70     |3.0   |964982400|
|1     |101    |5.0   |964980868|
|1     |110    |4.0   |964982176|
|1     |151    |5.0   |964984041|
|1     |157    |5.0   |964984100|
|1     |163    |5.0   |964983650|
|1     |216    |5.0   |964981208|
|1     |223    |3.0   |964980985|
|1     |231    |5.0   |964981179|
|1     |235    |4.0   |964980908|
|1     |260    |5.0   |964981680|
|1     |296    |3.0   |964982967|
|1     |316    |3.0   |964982310|
|1     |333    |5.0   |964981179|
|1     |349    |4.0   |964982563|
+------+-------+------+---------+
only showing top 20 rows

+------+-------+-----------------+----------+
|userId|movieId|tag              |timestamp |
+------+-------+-----------------+----------+
|2     |60756  |funn

In [7]:
#A.

rating1=ratings_df1.groupby(year("timestamp").alias("year")).agg(count('*').alias("no_of_ratings")).orderBy(col("year").desc())
#rating2=rating1.sort(col("year").desc())
rating1.show()

+----+-------------+
|year|no_of_ratings|
+----+-------------+
|2018|         6418|
|2017|         8198|
|2016|         6703|
|2015|         6616|
|2014|         1439|
|2013|         1664|
|2012|         4656|
|2011|         1690|
|2010|         2301|
|2009|         4158|
|2008|         4351|
|2007|         7114|
|2006|         4059|
|2005|         5813|
|2004|         3279|
|2003|         4014|
|2002|         3478|
|2001|         3922|
|2000|        10061|
|1999|         2439|
+----+-------------+
only showing top 20 rows



In [8]:
#B.

rating2=ratings_df1.groupby(month("timestamp").alias("month")).agg(count("rating").alias("rating_count")).orderBy("month")
rating2.show()

+-----+------------+
|month|rating_count|
+-----+------------+
|    1|        8684|
|    2|        7635|
|    3|        8880|
|    4|        7727|
|    5|       10883|
|    6|        8825|
|    7|        6950|
|    8|        9074|
|    9|        8510|
|   10|        7148|
|   11|        9676|
|   12|        6844|
+-----+------------+



In [9]:
#C.

rating3=ratings_df1.groupby(col("rating").alias("rating_distribution")).agg(count("rating").alias("rating_count")).orderBy(col("rating").desc())
rating3.show()

+-------------------+------------+
|rating_distribution|rating_count|
+-------------------+------------+
|                5.0|       13211|
|                4.5|        8551|
|                4.0|       26818|
|                3.5|       13136|
|                3.0|       20047|
|                2.5|        5550|
|                2.0|        7551|
|                1.5|        1791|
|                1.0|        2811|
|                0.5|        1370|
+-------------------+------------+



In [10]:
# movies_df.show(truncate=False)
# ratings_df1.show(truncate=False)
# tags_df1.show(truncate=False)

#D.

join_df=movies_df.join(ratings_df1,movies_df.movieId==ratings_df1.movieId,"left") \
                 .join(tags_df1,movies_df.movieId==tags_df1.movieId,"left") \
                 .select(movies_df.movieId,"title","genres",ratings_df1.userId,"rating",ratings_df1.timestamp,"tag").distinct()

join_df1=join_df.filter(col("rating").isNull() & col("tag").isNotNull()).select("title").distinct()
join_df1.count()       

18

In [11]:
#E.

join_df2=join_df.filter(col("rating").isNotNull() & col("tag").isNull()).select("title").distinct()
join_df2.count() 

8167

In [12]:
#F.

join_df3=join_df.filter(col("rating").isNotNull() & col("tag").isNull()).distinct()
#join_df3.show()

join_df4=join_df3.groupby("movieId","title").agg(avg("rating").alias("avg_rating"),count("rating").alias("count_rating")) \
                                            .orderBy(col("avg_rating").desc(),col("count_rating").desc())
#                                            .filter(col("count_title") > 30)
join_df4.show(10,truncate=False)

+-------+---------------------------------------------------------------------------------+----------+------------+
|movieId|title                                                                            |avg_rating|count_rating|
+-------+---------------------------------------------------------------------------------+----------+------------+
|78836  |Enter the Void (2009)                                                            |5.0       |2           |
|1151   |Lesson Faust (1994)                                                              |5.0       |2           |
|6442   |Belle époque (1992)                                                              |5.0       |2           |
|3473   |Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976)|5.0       |2           |
|99     |Heidi Fleiss: Hollywood Madam (1995)                                             |5.0       |2           |
|53     |Lamerica (1994)                                                

In [13]:
#G. 


total_tag_df= tags_df1.agg(count("tag").alias("avg_tags_per_movie")).collect()[0]['avg_tags_per_movie']
#toatl_tag_df.show()
display(total_tag_df)

total_tags_user = tags_df1.select("movieId").distinct().count()
display(total_tags_user)


3683

1572

In [14]:
#h.
#join_df.show()

# tag_movies_without_rate = join_df.filter(col("rating").isNull()).select("userId").distinct()
# tag_movies_without_rate.show()

users_tag_not_rated =movies_df.join(tags_df1,movies_df.movieId==tags_df1.movieId,"inner") \
                              .join(ratings_df1,movies_df.movieId==ratings_df1.movieId,"left").distinct()

# users_tag_not_rated.show()

users_tag_not_rated.printSchema()

users_tag_not_rated1 = users_tag_not_rated.filter(col("rating").isNull()).select(tags_df1.userId).distinct()

users_tag_not_rated1.show()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



+------+
|userId|
+------+
|   474|
|   318|
|   543|
|   288|
+------+



In [15]:
#I.

count_df =ratings_df1.agg(count("rating").alias("rating_user")).collect()[0]['rating_user']
print(count_df)
#count_df.show()
count_user =ratings_df1.select("userId").distinct().count()
print(count_user)
#count_user.show()

avg_rating =count_df/count_user
print(avg_rating)

100836
610
165.30491803278687


In [34]:
#J.

from pyspark.sql.window import Window

df1 =ratings_df1.join(movies_df,movies_df.movieId==ratings_df1.movieId,"inner") 
df2 = df1.withColumn("genre",explode(split("genres","\|"))).groupBy("rating","genre").count()

windf = Window.partitionBy("rating").orderBy(desc("count"))
rndf = df2.withColumn("rank",rank().over(windf)).filter(col("rank")==1).orderBy(col("rating").desc())
final_df = rndf.select("rating","genre")

final_df.show()

+------+------+
|rating| genre|
+------+------+
|   5.0| Drama|
|   4.5| Drama|
|   4.0| Drama|
|   3.5| Drama|
|   3.0|Comedy|
|   2.5|Comedy|
|   2.0|Comedy|
|   1.5|Comedy|
|   1.0|Comedy|
|   0.5|Comedy|
+------+------+



In [30]:
#K.
from pyspark.sql.window import Window

df1 =tags_df1.join(movies_df,movies_df.movieId==tags_df1.movieId,"inner") 

df2 = df1.withColumn("genre",explode(split("genres","\|"))).groupBy("genre","tag").count()

windf = Window.partitionBy("genre").orderBy(col("count").desc())
rankdf = df2.withColumn("rank",rank().over(windf)).filter(col("rank")==1).orderBy(desc("genre"))
final_Df = rankdf.select("genre","tag").groupBy("genre").agg(collect_list("tag").alias("tags_collection"))

print(final_Df)
final_Df.show(truncate=False)

DataFrame[genre: string, tags_collection: array<string>]
+------------------+-----------------------------------------------+
|genre             |tags_collection                                |
+------------------+-----------------------------------------------+
|Western           |[In Netflix queue]                             |
|War               |[Vietnam]                                      |
|Thriller          |[atmospheric]                                  |
|Sci-Fi            |[sci-fi]                                       |
|Romance           |[In Netflix queue]                             |
|Mystery           |[twist ending]                                 |
|Musical           |[Disney]                                       |
|IMAX              |[visually appealing, sci-fi, thought-provoking]|
|Horror            |[Stephen King, ghosts]                         |
|Film-Noir         |[In Netflix queue]                             |
|Fantasy           |[Disney]                  

In [18]:
#L.

join_df=movies_df.join(ratings_df1,movies_df.movieId==ratings_df1.movieId,"inner") \
                 .groupBy("title").agg(count("*").alias("cnt")).orderBy(col("cnt").desc())

join_df.show()

+--------------------+---+
|               title|cnt|
+--------------------+---+
| Forrest Gump (1994)|329|
|Shawshank Redempt...|317|
| Pulp Fiction (1994)|307|
|Silence of the La...|279|
|  Matrix, The (1999)|278|
|Star Wars: Episod...|251|
|Jurassic Park (1993)|238|
|   Braveheart (1995)|237|
|Terminator 2: Jud...|224|
|Schindler's List ...|220|
|   Fight Club (1999)|218|
|    Toy Story (1995)|215|
|Star Wars: Episod...|211|
|American Beauty (...|204|
|Usual Suspects, T...|204|
|Seven (a.k.a. Se7...|203|
|Independence Day ...|202|
|    Apollo 13 (1995)|201|
|Raiders of the Lo...|200|
|Lord of the Rings...|198|
+--------------------+---+
only showing top 20 rows



In [19]:
#m.

join_df=movies_df.join(ratings_df1,movies_df.movieId==ratings_df1.movieId,"inner")
join_df1 =join_df.groupBy("title").agg(avg("rating").alias("avg_rating"),count("rating").alias("count_rating"))

final_df =join_df1.filter(col("count_rating") >30).orderBy(col("avg_rating").desc()).distinct()
final_df.show(10)
final_df.printSchema()

+--------------------+------------------+------------+
|               title|        avg_rating|count_rating|
+--------------------+------------------+------------+
|       Psycho (1960)| 4.036144578313253|          83|
|Men in Black (a.k...| 3.487878787878788|         165|
|O Brother, Where ...|3.8085106382978724|          94|
|Snow White and th...| 3.616883116883117|          77|
|   Annie Hall (1977)|3.8706896551724137|          58|
|From Russia with ...|              3.75|          36|
|         Hook (1991)| 3.358490566037736|          53|
|Despicable Me (2010)|3.6842105263157894|          38|
|Kill Bill: Vol. 2...| 3.868181818181818|         110|
|Eternal Sunshine ...|4.1603053435114505|         131|
+--------------------+------------------+------------+
only showing top 10 rows

root
 |-- title: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- count_rating: long (nullable = false)

